In [357]:
import pandas as pd
import networkx as nx
import geopandas as gpd

In [358]:
parquet_file = "/Users/rachel1/Downloads/model-metadata-table.parquet"
df = pd.read_parquet(parquet_file, engine="pyarrow")

In [359]:
df

,LINKNO,DSLINKNO,strmOrder,USContArea,DSContArea,TDXHydroRegion,VPUCode,TopologicalOrder,LengthGeodesicMeters,TerminalLink,musk_k,musk_x
0,110007873,110009186,2,1.083060e+07,4.209271e+07,1020000010,102,177181,13384.058990,110013122,53536,0.25
1,110010498,110011810,2,1.096513e+07,2.296918e+07,1020000010,102,177182,2201.649965,110013122,8807,0.25
2,110005251,110019682,2,1.111977e+07,1.703163e+07,1020000010,102,177183,1972.054239,110030180,7888,0.25
3,110018371,110014434,2,1.076590e+07,3.751075e+07,1020000010,102,177184,5305.002476,110030180,21220,0.25
4,110009187,110011811,2,1.321197e+07,2.302526e+07,1020000010,102,177185,3587.784457,110030180,14351,0.25
...,...,...,...,...,...,...,...,...,...,...,...,...
6254099,820247408,820279661,9,1.666337e+12,1.666337e+12,8020008900,804,235109,3235.850704,820224554,3050,0.25
6254100,820279661,820281005,9,1.666343e+12,1.666378e+12,8020008900,804,235111,1513.076667,820224554,1426,0.25
6254101,820281005,820247403,9,1.666545e+12,1.666550e+12,8020008900,804,235112,3384.104153,820224554,3190,0.25
6254102,820247403,820224554,9,1.666565e+12,1.666577e+12,8020008900,804,235114,2834.209687,820224554,2671,0.25


In [360]:
keep = pd.read_json('/Volumes/EB406_T7_2/KEEP_POINTS.geojsonl.json', lines=True)

In [361]:
discard = [
    item['LINKNO']
    for item in keep['properties']
    if item.get('Keep') and item['Keep'].lower() == 'no'
]


In [362]:
trash = pd.read_json('/Users/rachel1/Downloads/trash_linknos.geojsonl.json', lines=True)
trash = [row['properties']['LINKNO'] for _, row in trash.iterrows()]
trash

[120772748,
 120752779,
 120714549,
 120684611,
 120717877,
 120694595,
 120671384,
 120661399,
 130085537,
 130129909,
 130099206,
 130152162,
 130131695,
 130140251,
 130131743,
 130116354,
 140406065,
 140672352,
 140685456,
 140579290,
 140580744,
 140343483,
 140481774,
 140362416,
 150230004,
 150305276,
 150340990,
 150196274,
 150368765,
 150370749,
 150380652,
 150382640,
 210071877,
 210066596,
 210244170,
 210252559,
 210219780,
 210249347,
 210253615,
 210272681,
 280438248,
 280439400,
 280372613,
 280366853,
 280411772,
 280372646,
 280388764,
 280438269,
 280463655,
 280406043,
 280423385,
 280396888,
 290829979,
 290503797,
 290527188,
 290543723,
 290551979,
 290765305,
 541726533,
 541422386,
 541425075,
 541484203,
 541750725,
 541493613,
 541168286,
 541687561,
 541563937,
 541078245,
 541074213,
 541142752,
 541597544,
 541714442,
 541494055,
 541573345,
 640372295,
 640353862,
 730428281,
 730461563,
 730512762,
 730467992,
 730501225,
 730464104,
 730470549,
 730

In [363]:
discard=trash+discard

First, we need to find all the terminal links that are big enough to not be thrown out.

In [364]:
terminal_links = df['TerminalLink'].unique()
terminal_links

array([110013122, 110030180, 110035430, ..., 820207539, 820178849,
       820032302], dtype=int32)

In [365]:
terminal_df = df[df['LINKNO'].isin(terminal_links)]
terminal_df

,LINKNO,DSLINKNO,strmOrder,USContArea,DSContArea,TDXHydroRegion,VPUCode,TopologicalOrder,LengthGeodesicMeters,TerminalLink,musk_k,musk_x
38829,110043348,-1,2,8.509690e+07,8.932893e+07,1020000010,102,218213,2216.288072,110043348,8865,0.25
39291,110114380,-1,3,4.883232e+07,5.248630e+07,1020000010,102,218748,3267.855034,110114380,13071,0.25
39437,110098657,-1,3,5.937556e+07,6.228312e+07,1020000010,102,218900,1700.605399,110098657,6802,0.25
39769,110098726,-1,2,4.038143e+07,6.176993e+07,1020000010,102,219267,13117.546548,110098726,52470,0.25
40339,110138157,-1,3,1.041439e+08,1.192091e+08,1020000010,102,219866,9482.048714,110138157,37928,0.25
...,...,...,...,...,...,...,...,...,...,...,...,...
6251347,820345786,-1,6,2.268494e+10,2.269639e+10,8020008900,804,231152,3326.388239,820345786,6459,0.01
6252453,820033653,-1,6,3.557212e+10,3.557848e+10,8020008900,804,232912,2153.713704,820033653,3881,0.25
6252697,820122377,-1,7,7.857015e+10,7.857386e+10,8020008900,804,233297,1613.976678,820122377,2542,0.25
6252819,820275660,-1,7,9.445369e+10,9.445495e+10,8020008900,804,233469,1945.855876,820275660,2971,0.25


In [366]:
large_terminal_df = terminal_df[terminal_df['DSContArea'] > 19000000000]
large_terminal_df

,LINKNO,DSLINKNO,strmOrder,USContArea,DSContArea,TDXHydroRegion,VPUCode,TopologicalOrder,LengthGeodesicMeters,TerminalLink,musk_k,musk_x
158376,110862667,-1,7,3.030946e+10,3.030953e+10,1020000010,103,342908,52.204567,110862667,96,0.25
158514,110173624,-1,7,2.491405e+10,2.491505e+10,1020000010,102,343062,1423.951058,110173624,2723,0.25
159310,110107575,-1,6,1.913517e+10,1.913849e+10,1020000010,101,343959,2361.643090,110107575,4723,0.25
159769,110323734,-1,6,2.420286e+10,2.420578e+10,1020000010,103,344479,2336.070849,110323734,4492,0.01
160845,110494343,-1,7,3.722115e+10,3.722608e+10,1020000010,101,345686,86.766151,110494343,155,0.01
...,...,...,...,...,...,...,...,...,...,...,...,...
6251347,820345786,-1,6,2.268494e+10,2.269639e+10,8020008900,804,231152,3326.388239,820345786,6459,0.01
6252453,820033653,-1,6,3.557212e+10,3.557848e+10,8020008900,804,232912,2153.713704,820033653,3881,0.25
6252697,820122377,-1,7,7.857015e+10,7.857386e+10,8020008900,804,233297,1613.976678,820122377,2542,0.25
6252819,820275660,-1,7,9.445369e+10,9.445495e+10,8020008900,804,233469,1945.855876,820275660,2971,0.25


In [367]:
split_terminal_df = terminal_df[terminal_df['DSContArea'] >= 100000000000]
split_terminal_df

,LINKNO,DSLINKNO,strmOrder,USContArea,DSContArea,TDXHydroRegion,VPUCode,TopologicalOrder,LengthGeodesicMeters,TerminalLink,musk_k,musk_x
165086,110552072,-1,8,1.764741e+11,1.764767e+11,1020000010,101,350459,4269.393858,110552072,5873,0.25
165320,110293680,-1,7,1.568916e+11,1.568994e+11,1020000010,101,350726,8289.913325,110293680,11627,0.25
165419,110858692,-1,7,1.359032e+11,1.359072e+11,1020000010,102,350873,363.245937,110858692,522,0.25
165472,110777430,-1,8,1.472204e+11,1.472536e+11,1020000010,103,350958,1164.198336,110777430,1651,0.01
165753,110407544,-1,9,7.415701e+11,7.415729e+11,1020000010,103,351257,3686.407102,110407544,3981,0.25
...,...,...,...,...,...,...,...,...,...,...,...,...
6084070,770490167,-1,7,1.175333e+11,1.175457e+11,7020047840,715,268446,22528.132623,770490167,33178,0.25
6084515,770390434,-1,8,5.042404e+11,5.042436e+11,7020047840,715,269004,2238.863092,770390434,2579,0.25
6177283,810389014,-1,7,1.230321e+11,1.231059e+11,8020000010,803,214050,10092.101697,810389014,14755,0.25
6177982,810543415,-1,9,9.229395e+11,9.229697e+11,8020000010,801,214856,11306.286945,810543415,11765,0.25


In [368]:
terminal_links_split = split_terminal_df['LINKNO'].unique()
terminal_links_split

array([110552072, 110293680, 110858692, 110777430, 110407544, 120772729,
       120831107, 120576786, 120712960, 120518766, 120734443, 130695715,
       130661995, 130796436, 130121500, 130761067, 140209596, 140556761,
       140391525, 140082968, 151316580, 150426280, 150516129, 151206905,
       151236700, 160144862, 160064246, 180335351, 180332556, 180422025,
       180424905, 210244179, 220725780, 220584104, 220463113, 230251953,
       230375729, 230523760, 240226641, 240205592, 280353126, 280366948,
       280378785, 280391139, 280400670, 280274809, 291091766, 290592616,
       290840992, 290636657, 290837320, 290679176, 291201685, 291153600,
       310335740, 310365513, 310393318, 320161466, 340093899, 360078283,
       410292506, 410239262, 410846189, 410681129, 420673932, 420063314,
       420543475, 420839057, 420711866, 430511608, 430552489, 430588315,
       430136929, 440602050, 441106878, 440325663, 440390305, 441060093,
       441072513, 460311853, 460279224, 460237273, 

In [369]:
links_to_split = df[df['TerminalLink'].isin(terminal_links_split)]
links_to_split

,LINKNO,DSLINKNO,strmOrder,USContArea,DSContArea,TDXHydroRegion,VPUCode,TopologicalOrder,LengthGeodesicMeters,TerminalLink,musk_k,musk_x
6997,110403336,110518792,2,1.196823e+07,5.597084e+07,1020000010,102,184923,9070.451190,110858692,36282,0.25
7176,110096331,110584397,2,1.244015e+07,2.943610e+07,1020000010,102,185104,11525.696844,110858692,46103,0.25
7181,110353483,110495181,2,1.069002e+07,3.534206e+07,1020000010,102,185109,11489.195779,110858692,45957,0.25
7186,110356107,110357419,2,1.065737e+07,1.818104e+07,1020000010,102,185114,1218.207389,110858692,4873,0.25
7248,110442700,110447948,2,1.065435e+07,2.045131e+07,1020000010,102,185177,1456.744718,110858692,5827,0.25
...,...,...,...,...,...,...,...,...,...,...,...,...
6254099,820247408,820279661,9,1.666337e+12,1.666337e+12,8020008900,804,235109,3235.850704,820224554,3050,0.25
6254100,820279661,820281005,9,1.666343e+12,1.666378e+12,8020008900,804,235111,1513.076667,820224554,1426,0.25
6254101,820281005,820247403,9,1.666545e+12,1.666550e+12,8020008900,804,235112,3384.104153,820224554,3190,0.25
6254102,820247403,820224554,9,1.666565e+12,1.666577e+12,8020008900,804,235114,2834.209687,820224554,2671,0.25


In [187]:
# Create a directed graph
G = nx.DiGraph()

# Add edges from the DataFrame
G.add_edges_from(df[['LINKNO', 'DSLINKNO']].dropna().itertuples(index=False, name=None))

print(f"Number of nodes: {G.number_of_nodes()}")
print(f"Number of edges: {G.number_of_edges()}")

Number of nodes: 6254105
Number of edges: 6254104


In [188]:
# Dictionary to store results for each terminal link
all_top_tributaries = {}

for start_link in terminal_links_split:
    print(f"Processing terminal link {start_link}...")
    current_link = start_link
    top_tributaries = {}  # Reset for each terminal link

    while True:
        # Find all upstream links that flow into the current link
        upstream_links = df[df['DSLINKNO'] == current_link]['LINKNO'].tolist()

        # If there's no merging (only one upstream), continue upstream
        if len(upstream_links) < 2:
            if not upstream_links:  # No more upstream nodes
                break
            current_link = upstream_links[0]
            continue

        # Get DSContArea for each upstream link
        upstream_data = links_to_split[links_to_split['LINKNO'].isin(upstream_links)]

        # If DSContArea is missing for any, continue safely
        if upstream_data.empty or len(upstream_data) < 2:
            break

        # Identify main branch and tributary
        main_branch = upstream_data.loc[upstream_data['DSContArea'].idxmax(), 'LINKNO']
        tributary = upstream_data.loc[upstream_data['DSContArea'].idxmin(), 'LINKNO']

        # Check if this tributary is among the top 4 largest encountered for this start_link
        if len(top_tributaries) < 4 or upstream_data['DSContArea'].min() > min(top_tributaries.values()):
            if len(top_tributaries) == 4:
                # Remove the smallest tributary to maintain top 4
                min_key = min(top_tributaries, key=top_tributaries.get)
                del top_tributaries[min_key]
            top_tributaries[tributary] = upstream_data['DSContArea'].min()

        # Continue upstream with the main branch
        current_link = main_branch

    # Store the results for this terminal link
    all_top_tributaries[start_link] = pd.DataFrame(
        list(top_tributaries.items()), columns=['LINKNO', 'DSContArea']
    )

# Print results
for terminal, df in all_top_tributaries.items():
    print(f"Top 4 tributaries for terminal {terminal}:\n{df}\n")


Processing terminal link 110552072...
Processing terminal link 110293680...
Processing terminal link 110858692...
Processing terminal link 110777430...
Processing terminal link 110407544...
Processing terminal link 120772729...
Processing terminal link 120831107...
Processing terminal link 120576786...
Processing terminal link 120712960...
Processing terminal link 120518766...
Processing terminal link 120734443...
Processing terminal link 130695715...
Processing terminal link 130661995...
Processing terminal link 130796436...
Processing terminal link 130121500...
Processing terminal link 130761067...
Processing terminal link 140209596...
Processing terminal link 140556761...
Processing terminal link 140391525...
Processing terminal link 140082968...
Processing terminal link 151316580...
Processing terminal link 150426280...
Processing terminal link 150516129...
Processing terminal link 151206905...
Processing terminal link 151236700...
Processing terminal link 160144862...
Processing t

In [370]:
# Get a unique list of all top tributaries
all_tributaries = set()

for df in all_top_tributaries.values():
    all_tributaries.update(df['LINKNO'].tolist())

# Convert to a sorted list (optional)
all_tributaries_list = sorted(all_tributaries)

print(all_tributaries_list)


[110293706, 110293711, 110354049, 110373735, 110386546, 110398348, 110435315, 110507466, 110525813, 110546824, 110585928, 110762915, 110769481, 110819332, 110832434, 110842898, 110848222, 110862664, 110923028, 110924310, 120517110, 120527076, 120545353, 120565386, 120595057, 120606670, 120607807, 120637841, 120639402, 120669645, 120671384, 120684611, 120684750, 120714549, 120714706, 120717970, 120732725, 120734383, 120752779, 120772674, 120774339, 120774346, 120782693, 120829261, 130085537, 130099206, 130129909, 130131695, 130285526, 130441860, 130620986, 130642245, 130644330, 130662014, 130681976, 130686028, 130688521, 130695376, 130706006, 130709353, 130721341, 130723080, 130738650, 130757640, 140343483, 140406065, 140537820, 140579290, 140591895, 140603709, 140610892, 140629840, 140631197, 140672352, 140680654, 140685297, 140703945, 140706840, 140719000, 140720433, 150230004, 150305276, 150340990, 150382640, 150508193, 150569706, 150901237, 150947732, 150958804, 151159386, 151177219

In [371]:
# Step 1: Find all DSLINKNO values where LINKNO is in all_tributaries_list
ds_links = links_to_split[links_to_split['LINKNO'].isin(all_tributaries_list)]['DSLINKNO'].unique()

# Step 2: Find all LINKNO values that have these DSLINKNO values
linked_linknos = links_to_split[links_to_split['DSLINKNO'].isin(ds_links)]['LINKNO'].tolist()

len(linked_linknos)

1036

730274253.0, 730257615.0, 730266619.0, 730333224.0, 730267899.0, 730253744.0, 730151278.0, 730171759.0, 730328054.0, 730331941.0,
730305063.0, 730321655.0, 730395965.0, 730401090.0, 730392287.0, 730389725.0

In [372]:
discard += [
    int(x) for x in [
        730274253.0, 730257615.0, 730266619.0, 730333224.0, 730267899.0,
        730253744.0, 730151278.0, 730171759.0, 730328054.0, 730331941.0,
        730305063.0, 730321655.0, 730395965.0, 730401090.0, 730392287.0,
        730389725.0
    ]
]

460088975, 460169892, 460308941, 460100748, 460101473, 460113252, 460199335, 460276556, 210071877, 210066596, 210244170,  210252559, 210219780, 210249347, 210253615, 210272681, 230261738, 230261741, 230297651, 230272638, 230261757, 230287852, 230307442, 230268266, 770590963, 770567016, 770556618, 770570200,770605530, 770446349, 770542057, 770560778, 770481700, 770650264, 770483780, 770625284, 770600329, 770486908, 770667944, 770588922, 750274424,750223563, 750276200, 750241320, 750250826, 750241319,  750262044, 750250232, 710304866, 710183875, 710279093,  710271149, 710281077, 710227525, 710314796, 710271141, 650215746,650183747, 650123773, 650216546, 650185347, 650126969, 650128573, 650140568

In [373]:
discard += [
    460088975, 460169892, 460308941, 460100748, 460101473, 460113252, 460199335, 460276556,
    210071877, 210066596, 210244170, 210252559, 210219780, 210249347, 210253615, 210272681,
    230261738, 230261741, 230297651, 230272638, 230261757, 230287852, 230307442, 230268266,
    770590963, 770567016, 770556618, 770570200, 770605530, 770446349, 770542057, 770560778,
    770481700, 770650264, 770483780, 770625284, 770600329, 770486908, 770667944, 770588922,
    750274424, 750223563, 750276200, 750241320, 750250826, 750241319, 750262044, 750250232,
    710304866, 710183875, 710279093, 710271149, 710281077, 710227525, 710314796, 710271141,
    650215746, 650183747, 650123773, 650216546, 650185347, 650126969, 650128573, 650140568, 410600536, 410553430, 360069329, 360075378, 360046908, 360064977, 360070787, 360056891, 360052729, 360052937, 110819332, 180329864, 180395145, 180405703, 650266760, 650252393,650298792, 650265164, 650321158, 650255561, 650305959,650262765, 640531664, 640533968, 621087515, 621071008,640531664, 640533968, 610453760, 610335809,290604846,290646136,290892379, 290782297, 290604858, 290571824, 290800188, 290787802,  410600536,410721288,
    650215746, 650183747, 650123773, 650216546, 650185347, 650126969, 650128573, 650140568, 410600536, 410553430, 360069329, 360075378, 360046908, 360064977, 360070787, 360056891, 360052729, 360052937, 110819332, 180329864, 180395145, 180405703, 650266760, 650252393,650298792, 650265164, 650321158, 650255561, 650305959,650262765, 640531664, 640533968, 621087515, 621071008,640531664, 640533968, 610453760, 610335809,290604846,290646136,290892379, 290782297, 290604858, 290571824, 290800188, 290787802,  410600536,410721288,410553430,410560588, 410763274
]


410600536, 410553430, 360069329, 360075378, 360046908, 360064977, 360070787, 360056891, 360052729, 360052937, 110819332, 180329864, 180395145, 180405703, 650266760, 650252393,650298792, 650265164, 650321158, 650255561, 650305959,650262765, 640531664, 640533968, 621087515, 621071008,640531664, 640533968, 610453760, 610335809

In [374]:
discard

[120772748,
 120752779,
 120714549,
 120684611,
 120717877,
 120694595,
 120671384,
 120661399,
 130085537,
 130129909,
 130099206,
 130152162,
 130131695,
 130140251,
 130131743,
 130116354,
 140406065,
 140672352,
 140685456,
 140579290,
 140580744,
 140343483,
 140481774,
 140362416,
 150230004,
 150305276,
 150340990,
 150196274,
 150368765,
 150370749,
 150380652,
 150382640,
 210071877,
 210066596,
 210244170,
 210252559,
 210219780,
 210249347,
 210253615,
 210272681,
 280438248,
 280439400,
 280372613,
 280366853,
 280411772,
 280372646,
 280388764,
 280438269,
 280463655,
 280406043,
 280423385,
 280396888,
 290829979,
 290503797,
 290527188,
 290543723,
 290551979,
 290765305,
 541726533,
 541422386,
 541425075,
 541484203,
 541750725,
 541493613,
 541168286,
 541687561,
 541563937,
 541078245,
 541074213,
 541142752,
 541597544,
 541714442,
 541494055,
 541573345,
 640372295,
 640353862,
 730428281,
 730461563,
 730512762,
 730467992,
 730501225,
 730464104,
 730470549,
 730

ADDITIONAL TO SPLIT: 180329864, 180395145, 180405703, 710194483, 720229576, 720233537, 720197926, 710938579, 720146346, 720164197, 710914988, 710797929,  710835637, 541868810,541699607, 541886341


In [375]:
additional_to_split = [
    180329864, 180395145, 180405703, 710194483, 720229576, 720233537, 720197926,
    710938579, 720146346, 720164197, 710914988, 710797929, 710835637,
    541868810, 541699607, 541886341
]

combined_linknos = list(
    set(linked_linknos) |
    set(terminal_links_split) |
    set(large_terminal_df['LINKNO']) |
    set(additional_to_split)
)


print(combined_linknos)
print(len(combined_linknos))

[640528384, 740483080, 220463113, 410763274, 151179272, 630292492, 470552584, 730333224, 120700972, 310681644, 420155438, 711000108, 120422445, 150196274, 230297651, 740175928, 621068345, 291311675, 470249536, 120684611, 640565315, 440500293, 120545353, 541732938, 760102985, 110936148, 610328663, 720175193, 470552666, 750055515, 151236700, 720183390, 541835359, 220471392, 621088867, 220475495, 650252393, 730501225, 150380652, 360075378, 770162807, 510103676, 460206205, 150524035, 130740362, 180330636, 140685456, 730480785, 110436498, 291311764, 541819030, 770650264, 140714143, 410501285, 280768678, 710271141, 410501288, 291123368, 710271149, 140685486, 240193715, 110538932, 291315892, 740253878, 280395961, 670085305, 290640059, 440328386, 650379459, 220528838, 810119367, 180424905, 120807627, 230478028, 151314635, 120684750, 541909196, 130695376, 750223563, 740667602, 340488407, 470507736, 760049883, 720187612, 220500192, 541573345, 280584420, 130789607, 310534376, 110305515, 170115307

Now I want to visualize what I have actually created

In [344]:
len(set(combined_linknos))

1848

In [267]:
filtered_links_to_split = links_to_split[links_to_split['LINKNO'].isin(combined_linknos)]
filtered_links_to_split = filtered_links_to_split.sort_values('TopologicalOrder')
ordered = filtered_links_to_split['LINKNO'].to_list()

In [235]:
nexus_points = gpd.read_file("/Users/rachel1/Downloads/global_nexus.gpkg")

KeyboardInterrupt: 

In [20]:
global_streams = gpd.read_file('/Users/rachel1/Downloads/global_streams_simplified (1).gpkg')

30,32, 64, 73, 115,

In [280]:
filtered_combined_linknos = [ln for ln in combined_linknos if ln not in discard]
len(filtered_combined_linknos)

2085

In [376]:
filtered_combined_linknos = [ln for ln in combined_linknos if ln not in discard]

filtered_links_to_split = large_terminal_df[
    large_terminal_df['LINKNO'].isin(filtered_combined_linknos)
].sort_values('TopologicalOrder')
ordered = filtered_links_to_split['LINKNO'].to_list()
filtered_ordered = [ln for ln in ordered if ln not in discard]

In [377]:
filtered_links_to_split

,LINKNO,DSLINKNO,strmOrder,USContArea,DSContArea,TDXHydroRegion,VPUCode,TopologicalOrder,LengthGeodesicMeters,TerminalLink,musk_k,musk_x
4319565,570049746,-1,6,2.077377e+10,2.077740e+10,5020082270,514,35319,2939.619129,570049746,5798,0.25
2550423,360048079,-1,8,6.662204e+10,6.662204e+10,3020024310,305,44074,24.718540,360048079,40,0.01
2550529,360078283,-1,8,1.346679e+11,1.346752e+11,3020024310,305,44231,588.919113,360078283,847,0.01
5243721,670069168,-1,6,2.661609e+10,2.662282e+10,6020029280,614,52926,4571.187311,670069168,8641,0.25
5243864,670061698,-1,6,2.177111e+10,2.178146e+10,6020029280,614,53086,6631.101698,670061698,12977,0.25
...,...,...,...,...,...,...,...,...,...,...,...,...
4304736,541886341,-1,7,6.523872e+10,6.524704e+10,5020049720,504,933055,5030.421158,541886341,8180,0.25
4304760,541692942,-1,7,4.695039e+10,4.695039e+10,5020049720,509,933083,48.742275,541692942,84,0.25
4304834,541500780,-1,8,1.035017e+11,1.035045e+11,5020049720,509,933157,3423.779207,541500780,5149,0.25
4305008,541815140,-1,9,5.472693e+11,5.472693e+11,5020049720,513,933384,139.626656,541815140,159,0.01


In [378]:
len(filtered_ordered)

666

In [379]:
subgraph = G.subgraph(filtered_combined_linknos)

# Step 2: Get the topological order for the subgraph
topological_order = list(nx.topological_sort(subgraph))


In [380]:
topological_order

[630233089,
 180332546,
 110446595,
 750073858,
 110389255,
 740483080,
 220463113,
 151179272,
 470552584,
 630292492,
 610240533,
 110323734,
 150788119,
 280758311,
 120700972,
 310681644,
 711000108,
 120422445,
 140429367,
 740175928,
 621068345,
 460286007,
 291311675,
 151316539,
 120711228,
 470249536,
 470530112,
 120772674,
 110585928,
 120545353,
 760102985,
 130717768,
 420063314,
 610328663,
 151255127,
 720175193,
 470552666,
 750055515,
 151236700,
 621160536,
 760658012,
 541835359,
 220471392,
 740329561,
 150849625,
 621088867,
 540375136,
 740436067,
 220475495,
 140540008,
 280750188,
 621000813,
 120518766,
 440965230,
 540514417,
 630335603,
 820033653,
 770162807,
 120772729,
 180316282,
 710441083,
 510103676,
 150524035,
 760359044,
 520165508,
 540854406,
 130740362,
 180330636,
 320383120,
 730480785,
 110436498,
 621017234,
 291311764,
 480381075,
 541819030,
 770164882,
 740341914,
 140714143,
 280768678,
 140617894,
 291123368,
 220584104,
 460261547,
 140

In [381]:
downstream_associations = {}
i = 0
#filtered_ordered = filtered_ordered[::-1]

# Step 3: Process the links in topological order
for link in filtered_ordered:
    i = i + 1
    print(f"Processing link {link}... ({i}/{len(topological_order)})")

    # Find all upstream nodes (including the node itself)
    upstream_nodes = nx.ancestors(G, link)
    upstream_nodes.add(link)  # Include the current link itself

    # For each upstream link, check if it's associated with any downstream link
    for upstream in upstream_nodes:
        if upstream not in downstream_associations:
            # If not already associated, assign the current downstream link
            downstream_associations[upstream] = link

# Step 4: Create a DataFrame from the associations
downstream_df = pd.DataFrame(downstream_associations.items(), columns=['LINKNO', 'Downstream_Link'])

# Step 5: Merge with links_to_split
links_to_split = pd.merge(large_terminal_df, downstream_df, on='LINKNO', how='left')

print(links_to_split)

Processing link 570049746... (1/1043)
Processing link 360048079... (2/1043)
Processing link 360078283... (3/1043)
Processing link 670069168... (4/1043)
Processing link 670061698... (5/1043)
Processing link 670085305... (6/1043)
Processing link 170105279... (7/1043)
Processing link 170125943... (8/1043)
Processing link 170130325... (9/1043)
Processing link 170114945... (10/1043)
Processing link 170131957... (11/1043)
Processing link 170115307... (12/1043)
Processing link 250104868... (13/1043)
Processing link 250160324... (14/1043)
Processing link 250089969... (15/1043)
Processing link 250050536... (16/1043)
Processing link 510103676... (17/1043)
Processing link 510103884... (18/1043)
Processing link 510094150... (19/1043)
Processing link 490083142... (20/1043)
Processing link 530183539... (21/1043)
Processing link 530256727... (22/1043)
Processing link 530217959... (23/1043)
Processing link 530161922... (24/1043)
Processing link 530177103... (25/1043)
Processing link 530183644... (26/1

In [321]:
large_terminal_df

,LINKNO,DSLINKNO,strmOrder,USContArea,DSContArea,TDXHydroRegion,VPUCode,TopologicalOrder,LengthGeodesicMeters,TerminalLink,musk_k,musk_x
157745,110726093,-1,6,1.494225e+10,1.494388e+10,1020000010,102,342198,1040.026310,110726093,2167,0.25
157974,110853412,-1,6,1.046334e+10,1.046575e+10,1020000010,102,342457,104.155824,110853412,230,0.01
158007,110335446,-1,6,1.506352e+10,1.506352e+10,1020000010,103,342490,17.425285,110335446,36,0.01
158376,110862667,-1,7,3.030946e+10,3.030953e+10,1020000010,103,342908,52.204567,110862667,96,0.25
158514,110173624,-1,7,2.491405e+10,2.491505e+10,1020000010,102,343062,1423.951058,110173624,2723,0.25
...,...,...,...,...,...,...,...,...,...,...,...,...
6251347,820345786,-1,6,2.268494e+10,2.269639e+10,8020008900,804,231152,3326.388239,820345786,6459,0.01
6252453,820033653,-1,6,3.557212e+10,3.557848e+10,8020008900,804,232912,2153.713704,820033653,3881,0.25
6252697,820122377,-1,7,7.857015e+10,7.857386e+10,8020008900,804,233297,1613.976678,820122377,2542,0.25
6252819,820275660,-1,7,9.445369e+10,9.445495e+10,8020008900,804,233469,1945.855876,820275660,2971,0.25


In [383]:
len(downstream_df['Downstream_Link'].unique())

666

In [315]:
links_to_split = links_to_split[['LINKNO', 'Downstream_Link']]


In [289]:
links_to_split

,LINKNO,Downstream_Link
0,110331512,110331512.0
1,110237934,110237934.0
2,110320048,110320048.0
3,110427316,110427316.0
4,110370929,110370929.0
...,...,...
1699,820345786,820345786.0
1700,820033653,820033653.0
1701,820122377,820122377.0
1702,820275660,820275660.0


In [382]:
downstream_df.to_csv('/Users/rachel1/Downloads/downstream22.csv', index=False)

In [350]:
downstream_df

,LINKNO,Downstream_Link
0,541327363,540514417
1,541065224,540514417
2,541327369,540514417
3,541065227,540514417
4,541327372,540514417
...,...,...
4849069,570037418,570049746
4849070,570043524,570049746
4849071,570077310,570049746
4849072,570075275,570049746


In [105]:
# Step 6: Merge global_streams with downstream_df on LINKNO
global_streams_link = pd.merge(global_streams, downstream_df, on='LINKNO', how='left')

# Print the updated global_streams
print(global_streams_link)

,LINKNO,DSLINKNO,strmOrder,USContArea,DSContArea,TDXHydroRegion,VPUCode,TopologicalOrder,LengthGeodesicMeters,TerminalLink,musk_k,musk_x,Downstream_Link
0,110403336,110518792,2,1.196823e+07,5.597084e+07,1020000010,102,184923,9070.451190,110858692,36282,0.25,110832433
1,110096331,110584397,2,1.244015e+07,2.943610e+07,1020000010,102,185104,11525.696844,110858692,46103,0.25,110832433
2,110353483,110495181,2,1.069002e+07,3.534206e+07,1020000010,102,185109,11489.195779,110858692,45957,0.25,110832433
3,110356107,110357419,2,1.065737e+07,1.818104e+07,1020000010,102,185114,1218.207389,110858692,4873,0.25,110832433
4,110442700,110447948,2,1.065435e+07,2.045131e+07,1020000010,102,185177,1456.744718,110858692,5827,0.25,110832433
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3565388,820247408,820279661,9,1.666337e+12,1.666337e+12,8020008900,804,235109,3235.850704,820224554,3050,0.25,820224554
3565389,820279661,820281005,9,1.666343e+12,1.666378e+12,8020008900,804,235111,1513.076667,820224554,1426,0.25,820224554
3565390,820281005,820247403,9,1.666545e+12,1.666550e+12,8020008900,804,235112,3384.104153,820224554,3190,0.25,820224554
3565391,820247403,820224554,9,1.666565e+12,1.666577e+12,8020008900,804,235114,2834.209687,820224554,2671,0.25,820224554


In [244]:
terminal = global_streams[global_streams['LINKNO'].isin(terminal_df['LINKNO'].unique())]
terminal.to_file('/Users/rachel1/Downloads/terminal.gpkg', driver='GPKG')

In [120]:
global_streams_link.to_file('/Users/rachel1/Downloads/global_streams_link.gpkg', driver='GPKG')

In [203]:
filtered_global_streams = global_streams[global_streams['LINKNO'].isin(links_to_split['Downstream_Link'].unique())]
filtered_global_streams.to_file('/Users/rachel1/Downloads/filtered_global_streams22.gpkg', driver='GPKG')

In [206]:
130085537 in discard


False